In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
import os, glob

print("True.csv hits:", glob.glob('/content/drive/MyDrive/**/True.csv', recursive=True)[:10])
print("Fake.csv hits:", glob.glob('/content/drive/MyDrive/**/Fake.csv', recursive=True)[:10])

TRUE_PATH = '/content/drive/MyDrive/True.csv'
FAKE_PATH = '/content/drive/MyDrive/Fake.csv'

print("TRUE exists? ", os.path.exists(TRUE_PATH), "->", TRUE_PATH)
print("FAKE exists? ", os.path.exists(FAKE_PATH), "->", FAKE_PATH)


True.csv hits: ['/content/drive/MyDrive/True.csv']
Fake.csv hits: ['/content/drive/MyDrive/Fake.csv']
TRUE exists?  True -> /content/drive/MyDrive/True.csv
FAKE exists?  True -> /content/drive/MyDrive/Fake.csv


In [ ]:
from pathlib import Path

TRUE_CSV = Path(TRUE_PATH)
FAKE_CSV = Path(FAKE_PATH)

assert TRUE_CSV.exists(), f"Missing: {TRUE_CSV}"
assert FAKE_CSV.exists(), f"Missing: {FAKE_CSV}"

print("Using:", TRUE_CSV, FAKE_CSV)


Using: /content/drive/MyDrive/True.csv /content/drive/MyDrive/Fake.csv


**Imports and basic settings**:

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


**Load ISOT Fake/True dataset:**

In [ ]:
def load_isot(fake_path, true_path):
    fake_df = pd.read_csv(fake_path)
    true_df = pd.read_csv(true_path)

    fake_df["label"] = "fake"
    true_df["label"] = "true"

    # Combine two dataframes
    df = pd.concat([fake_df, true_df], ignore_index=True)

    # One text column = title + body
    df["text_full"] = df["title"].fillna("") + " " + df["text"].fillna("")

    # Drop exact duplicate articles
    df = df.drop_duplicates(subset=["title", "text"])

    # Keep only the columns we care about now
    df = df[["text_full", "label", "subject", "date"]]

    return df


# use the Path objects we already created
df = load_isot(FAKE_CSV, TRUE_CSV)

print(df.head())
print("\nLabel counts:")
print(df["label"].value_counts())


                                           text_full label subject  \
0   Donald Trump Sends Out Embarrassing New Year’...  fake    News   
1   Drunk Bragging Trump Staffer Started Russian ...  fake    News   
2   Sheriff David Clarke Becomes An Internet Joke...  fake    News   
3   Trump Is So Obsessed He Even Has Obama’s Name...  fake    News   
4   Pope Francis Just Called Out Donald Trump Dur...  fake    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  

Label counts:
label
true    21197
fake    17908
Name: count, dtype: int64


**Train / validation / test split:**

In [ ]:
X = df["text_full"].values
y = df["label"].values

from sklearn.model_selection import train_test_split

# Train 70% vs temp 30%
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=RANDOM_STATE
)

# Validation 15% vs Test 15%
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=RANDOM_STATE
)

print("Train size:", len(X_train))
print("Valid size:", len(X_valid))
print("Test size:", len(X_test))


Train size: 27373
Valid size: 5866
Test size: 5866


**Preprocessing and vocabulary:**

In [ ]:
import re
TOKEN_PATTERN = re.compile(r"[a-z]+")   # regex pattern to extract alphabetic tokens only (a–z)

def preprocess_text(text):
    """
    Lowercase, remove URLs, keep alphabetic tokens only.
    """
    text = text.lower()   # convert entire text to lowercase for consistency
    text = re.sub(r"http\S+|www\.\S+", " ", text)   # remove URLs and replace them with spaces
    tokens = TOKEN_PATTERN.findall(text)   # extract alphabetic tokens using the regex pattern
    return tokens    # return list of cleaned tokens


def build_vocabulary(texts, min_freq=5):
    """
    Build a word -> index dictionary from training texts.
    Only keep words that appear at least min_freq times.
    """
    freq = {}   # dictionary to count token frequencies across all training texts

    for t in texts:
        tokens = preprocess_text(t)   # preprocess each document
        for tok in tokens:
            freq[tok] = freq.get(tok, 0) + 1   # increment count or initialize to 1

    vocab = {}   # final vocabulary mapping token -> index
    idx = 0      # index counter
    for word, count in freq.items():
        if count >= min_freq:     # keep only words that occur enough times
            vocab[word] = idx     # assign index to the word
            idx += 1              # increment index for next word

    return vocab   # return constructed vocabulary dictionary


def vectorize_tokens(tokens, vocab):
    """
    Convert list of tokens into a count vector.
    """
    vec = np.zeros(len(vocab), dtype=np.int32)   # initialize vector of zeros, length = vocab size
    for tok in tokens:
        if tok in vocab:         # check if token exists in vocabulary
            j = vocab[tok]       # get index of the token
            vec[j] += 1          # increment count in the corresponding position
    return vec    # return the bag-of-words count vector


# Build vocabulary from the training set
vocab = build_vocabulary(X_train, min_freq=5)   # construct vocabulary using training data
print("Vocabulary size:", len(vocab))           # print number of unique tokens kept


Vocabulary size: 35464


**From-scratch Multinomial Naive Bayes:**

In [ ]:
class MultinomialNBScratch:
    """
    Simple Multinomial Naive Bayes for text classification.
    """

    def __init__(self, alpha=1.0):
        self.alpha = alpha                      # Laplace smoothing parameter
        self.classes_ = None                    # stores the class labels (e.g., ["fake", "true"])
        self.class_priors_ = None               # P(class) for each class
        self.feature_log_prob_ = None           # log P(word | class) matrix
        self.vocab_ = None                      # reference to the vocabulary used

    def fit(self, texts, labels, vocab):
        """
        Train the Naive Bayes model on the given texts and labels.
        """
        self.vocab_ = vocab                     # save the vocabulary
        n_docs = len(texts)                     # number of documents in the training set

        # find unique classes and map labels to numeric indices
        self.classes_, y_indices = np.unique(labels, return_inverse=True)
        n_classes = len(self.classes_)          # number of classes (2 in our case)
        n_features = len(vocab)                 # size of vocabulary

        # initialize word count matrix [num_classes x vocab_size]
        class_word_counts = np.zeros((n_classes, n_features), dtype=np.int64)
        class_doc_counts = np.zeros(n_classes, dtype=np.int64)   # document count per class

        # loop over every training document
        for i, text in enumerate(texts):
            c_idx = y_indices[i]                # index of the class for this document
            class_doc_counts[c_idx] += 1        # count how many docs belong to each class

            tokens = preprocess_text(text)      # preprocess text into tokens
            vec = vectorize_tokens(tokens, vocab)   # convert tokens into a count vector
            class_word_counts[c_idx] += vec     # add counts to this class's word totals

        # compute prior probabilities P(class)
        self.class_priors_ = class_doc_counts / n_docs

        # apply Laplace smoothing to word counts
        alpha = self.alpha
        smoothed = class_word_counts + alpha

        # sum of word counts for each class (needed for normalization)
        class_totals = smoothed.sum(axis=1).reshape(-1, 1)

        # compute conditional probabilities P(word | class) in log-space
        self.feature_log_prob_ = np.log(smoothed / class_totals)

    def _log_posterior(self, text):
        """
        Compute log posterior probabilities for each class for a single document.
        """
        tokens = preprocess_text(text)                 # preprocess text
        vec = vectorize_tokens(tokens, self.vocab_)    # convert to count vector

        # log likelihood: sum(count * log P(word|class)) over all words
        log_likelihood = (self.feature_log_prob_ * vec).sum(axis=1)

        # log prior: log(P(class))
        log_prior = np.log(self.class_priors_)

        # log posterior for each class
        return log_prior + log_likelihood

    def predict(self, texts):
        """
        Predict class labels for a list of texts.
        """
        preds = []
        for t in texts:
            log_post = self._log_posterior(t)          # compute log posteriors
            c_idx = np.argmax(log_post)                # choose class with highest score
            preds.append(self.classes_[c_idx])         # map index back to class label
        return np.array(preds)


**Small subset experiment:**

In [ ]:
train_subset_size = 1000
test_subset_size = 300

X_train_small = X_train[:train_subset_size]
y_train_small = y_train[:train_subset_size]
X_test_small = X_test[:test_subset_size]
y_test_small = y_test[:test_subset_size]

vocab_small = build_vocabulary(X_train_small, min_freq=3)
print("Small vocab size:", len(vocab_small))

nb_small = MultinomialNBScratch(alpha=1.0)
nb_small.fit(X_train_small, y_train_small, vocab_small)

y_pred_small = nb_small.predict(X_test_small)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

acc_small = accuracy_score(y_test_small, y_pred_small)
print(f"Subset accuracy (scratch NB) = {acc_small:.4f}")

print("\nClassification report (subset):")
print(classification_report(y_test_small, y_pred_small))

cm_small = confusion_matrix(y_test_small, y_pred_small, labels=["fake", "true"])
print("Confusion matrix (rows=actual, cols=pred, order=[fake, true]):\n", cm_small)


Small vocab size: 9752
Subset accuracy (scratch NB) = 0.9533

Classification report (subset):
              precision    recall  f1-score   support

        fake       0.94      0.96      0.95       139
        true       0.97      0.94      0.96       161

    accuracy                           0.95       300
   macro avg       0.95      0.95      0.95       300
weighted avg       0.95      0.95      0.95       300

Confusion matrix (rows=actual, cols=pred, order=[fake, true]):
 [[134   5]
 [  9 152]]


**Full train experiment:**

In [ ]:
vocab_full = build_vocabulary(X_train, min_freq=5)
print("Full vocab size:", len(vocab_full))

nb_full = MultinomialNBScratch(alpha=1.0)
nb_full.fit(X_train, y_train, vocab_full)

y_pred_test = nb_full.predict(X_test)
acc_test = accuracy_score(y_test, y_pred_test)

print(f"Test accuracy (scratch NB) = {acc_test:.4f}")
print("\nClassification report (full test):")
print(classification_report(y_test, y_pred_test))

cm_full = confusion_matrix(y_test, y_pred_test, labels=["fake", "true"])
print("Confusion matrix (rows=actual, cols=pred, order=[fake, true]):\n", cm_full)


Full vocab size: 35464
Test accuracy (scratch NB) = 0.9535

Classification report (full test):
              precision    recall  f1-score   support

        fake       0.95      0.95      0.95      2687
        true       0.96      0.96      0.96      3179

    accuracy                           0.95      5866
   macro avg       0.95      0.95      0.95      5866
weighted avg       0.95      0.95      0.95      5866

Confusion matrix (rows=actual, cols=pred, order=[fake, true]):
 [[2552  135]
 [ 138 3041]]


In [ ]:
# Model 2: Logistic Regression with TF-IDF features

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. Build TF-IDF features from training text and apply to val/test
tfidf = TfidfVectorizer(
    lowercase=True,
    stop_words='english',   # you can remove this if you don't want stopword removal
    min_df=5,               # ignore very rare words
    ngram_range=(1, 2)      # unigrams + bigrams
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_valid_tfidf = tfidf.transform(X_valid)
X_test_tfidf  = tfidf.transform(X_test)

print("TF-IDF shapes:")
print("  train:", X_train_tfidf.shape)
print("  valid:", X_valid_tfidf.shape)
print("  test :", X_test_tfidf.shape)

# 2. Train Logistic Regression classifier
log_reg = LogisticRegression(
    max_iter=1000,
    n_jobs=-1
)

log_reg.fit(X_train_tfidf, y_train)

# 3. Evaluate on validation set
y_valid_pred = log_reg.predict(X_valid_tfidf)
print("\nValidation performance (Logistic Regression):")
print("Accuracy:", accuracy_score(y_valid, y_valid_pred))
print(classification_report(y_valid, y_valid_pred))

# 4. Final evaluation on test set
y_test_pred = log_reg.predict(X_test_tfidf)
print("\nTest performance (Logistic Regression):")
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

print("Confusion matrix (test):")
print(confusion_matrix(y_test, y_test_pred))


TF-IDF shapes:
  train: (27373, 179817)
  valid: (5866, 179817)
  test : (5866, 179817)

Validation performance (Logistic Regression):
Accuracy: 0.9856801909307876
              precision    recall  f1-score   support

        fake       0.99      0.98      0.98      2686
        true       0.98      0.99      0.99      3180

    accuracy                           0.99      5866
   macro avg       0.99      0.99      0.99      5866
weighted avg       0.99      0.99      0.99      5866


Test performance (Logistic Regression):
Accuracy: 0.9858506648482782
              precision    recall  f1-score   support

        fake       0.99      0.98      0.98      2687
        true       0.98      0.99      0.99      3179

    accuracy                           0.99      5866
   macro avg       0.99      0.99      0.99      5866
weighted avg       0.99      0.99      0.99      5866

Confusion matrix (test):
[[2626   61]
 [  22 3157]]


In [ ]:
# Model 3: Linear SVM (LinearSVC) with the same TF-IDF features

from sklearn.svm import LinearSVC

svm_clf = LinearSVC()

# 1. Train on training TF-IDF features
svm_clf.fit(X_train_tfidf, y_train)

# 2. Evaluate on validation set
y_valid_pred_svm = svm_clf.predict(X_valid_tfidf)
print("\nValidation performance (LinearSVC):")
print("Accuracy:", accuracy_score(y_valid, y_valid_pred_svm))
print(classification_report(y_valid, y_valid_pred_svm))

# 3. Final evaluation on test set
y_test_pred_svm = svm_clf.predict(X_test_tfidf)
print("\nTest performance (LinearSVC):")
print("Accuracy:", accuracy_score(y_test, y_test_pred_svm))
print(classification_report(y_test, y_test_pred_svm))

print("Confusion matrix (test, LinearSVC):")
print(confusion_matrix(y_test, y_test_pred_svm))



Validation performance (LinearSVC):
Accuracy: 0.9938629389703375
              precision    recall  f1-score   support

        fake       1.00      0.99      0.99      2686
        true       0.99      1.00      0.99      3180

    accuracy                           0.99      5866
   macro avg       0.99      0.99      0.99      5866
weighted avg       0.99      0.99      0.99      5866


Test performance (LinearSVC):
Accuracy: 0.9933515172178656
              precision    recall  f1-score   support

        fake       1.00      0.99      0.99      2687
        true       0.99      1.00      0.99      3179

    accuracy                           0.99      5866
   macro avg       0.99      0.99      0.99      5866
weighted avg       0.99      0.99      0.99      5866

Confusion matrix (test, LinearSVC):
[[2655   32]
 [   7 3172]]


In [ ]:
# ============================================================
# Model 4: RNN (Bidirectional LSTM) on raw text
# ============================================================

# 2. Tokenize the raw text and create padded sequences

MAX_VOCAB = 30000   # max number of words in the vocabulary
MAX_LEN   = 300     # max number of tokens per article

# Fit tokenizer on training text ONLY
tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

# Convert text -> integer sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_valid_seq = tokenizer.texts_to_sequences(X_valid)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

# Pad / truncate to fixed length
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
X_valid_pad = pad_sequences(X_valid_seq, maxlen=MAX_LEN, padding="post", truncating="post")
X_test_pad  = pad_sequences(X_test_seq,  maxlen=MAX_LEN, padding="post", truncating="post")

print("Padded shapes:")
print("  train:", X_train_pad.shape)
print("  valid:", X_valid_pad.shape)
print("  test :", X_test_pad.shape)


# 1. Encode labels ("fake" / "true") as 0 / 1 for Keras
label_to_id = {"fake": 0, "true": 1}

y_train_int = np.array([label_to_id[y] for y in y_train])
y_valid_int = np.array([label_to_id[y] for y in y_valid])
y_test_int  = np.array([label_to_id[y] for y in y_test])

print("Label encoding check (train):", np.bincount(y_train_int))


# 3. Build the Bidirectional LSTM model

vocab_size    = min(MAX_VOCAB, len(tokenizer.word_index) + 1)
embedding_dim = 100

rnn_model = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=MAX_LEN
    ),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")   # binary output
])

rnn_model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

rnn_model.summary()


# 4. Train the RNN with early stopping on the validation set

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=2,
    restore_best_weights=True
)

history = rnn_model.fit(
    X_train_pad, y_train_int,
    validation_data=(X_valid_pad, y_valid_int),
    epochs=4,
    batch_size=128,
    callbacks=[early_stop],
    verbose=1
)

# 5. Evaluate on the test set and compare with NB / Logistic / SVM

test_loss, test_acc = rnn_model.evaluate(X_test_pad, y_test_int, verbose=0)
print(f"Test accuracy (RNN, BiLSTM) = {test_acc:.4f}")

y_prob = rnn_model.predict(X_test_pad).ravel()
y_pred = (y_prob >= 0.5).astype(int)

print("\nClassification report (RNN, BiLSTM):")
print(classification_report(y_test_int, y_pred, target_names=["fake", "true"]))

print("Confusion matrix (RNN, BiLSTM):")
print(confusion_matrix(y_test_int, y_pred))


Padded shapes:
  train: (27373, 300)
  valid: (5866, 300)
  test : (5866, 300)
Label encoding check (train): [12535 14838]


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/4
214/214 ━━━━━━━━━━━━━━━━━━━━ 280s 1s/step - accuracy: 0.8605 - loss: 0.2819 - val_accuracy: 0.9986 - val_loss: 0.0043
Epoch 2/4
214/214 ━━━━━━━━━━━━━━━━━━━━ 272s 1s/step - accuracy: 0.9997 - loss: 0.0029 - val_accuracy: 0.9988 - val_loss: 0.0051
Epoch 3/4
214/214 ━━━━━━━━━━━━━━━━━━━━ 268s 1s/step - accuracy: 0.9999 - loss: 0.0011 - val_accuracy: 0.9980 - val_loss: 0.0076
Test accuracy (RNN, BiLSTM) = 0.9980
184/184 ━━━━━━━━━━━━━━━━━━━━ 17s 91ms/step

Classification report (RNN, BiLSTM):
              precision    recall  f1-score   support

        fake       1.00      1.00      1.00      2687
        true       1.00      1.00      1.00      3179

    accuracy                           1.00      5866
   macro avg       1.00      1.00      1.00      5866
weighted avg       1.00      1.00      1.00      5866

Confusion matrix (RNN, BiLSTM):
[[2679    8]
 [   4 3175]]
